In [1]:
from qiskit import QuantumCircuit
from qiskit.compiler import transpile
from qiskit_aer import AerSimulator
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit_aer.noise import depolarizing_error, NoiseModel
from qiskit.quantum_info import SparsePauliOp

from qiskit.primitives import BackendEstimator

from zne import zne, ZNEStrategy
from zne.noise_amplification import *
from zne.extrapolation import *

import numpy as np
import matplotlib.pyplot as plt

import json, os, pickle, random
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn

import qiskit
from qiskit import QuantumCircuit, execute
from qiskit.compiler import transpile
from qiskit_aer import AerSimulator, QasmSimulator
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit.quantum_info import SparsePauliOp, Operator
from qiskit.circuit.library import CXGate, RXGate, IGate, ZGate
from qiskit.providers.fake_provider import FakeMontreal, FakeLima

from blackwater.data.utils import (
    generate_random_pauli_sum_op,
    create_estimator_meas_data,
    circuit_to_graph_data_json,
    get_backend_properties_v1,
    encode_pauli_sum_op,
    create_meas_data_from_estimators
)

from mlp import MLP1, MLP2, MLP3, encode_data

from mbd_utils import cal_z_exp, generate_disorder, construct_mbl_circuit, calc_imbalance, modify_and_add_noise_to_model

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
backend = FakeLima()
properties = get_backend_properties_v1(backend)

## Local
backend_ideal = QasmSimulator() # Noiseless
backend_noisy = AerSimulator.from_backend(FakeLima()) # Noisy

run_config_ideal = {'shots': 10000, 'backend': backend_ideal, 'name': 'ideal'}
run_config_noisy = {'shots': 10000, 'backend': backend_noisy, 'name': 'noisy'}

In [3]:
def fix_random_seed(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    print(f'random seed fixed to {seed}')

In [4]:
def load_circuits(data_dir, f_ext='.json'):
    circuits = []
    ideal_exp_vals = []
    noisy_exp_vals = []
    data_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith(f_ext)]
    for data_file in tqdm(data_files, leave=True):
        if f_ext == '.json':
            for entry in json.load(open(data_file, 'r')):
                circuits.append(QuantumCircuit.from_qasm_str(entry['circuit']))
                ideal_exp_vals.append(entry['ideal_exp_value'])
                noisy_exp_vals.append(entry['noisy_exp_values'])
        elif f_ext == '.pk':
            for entry in pickle.load(open(data_file, 'rb')):
                circuits.append(entry['circuit'])
                ideal_exp_vals.append(entry['ideal_exp_value'])
                noisy_exp_vals.append(entry['noisy_exp_values'])
    return circuits, ideal_exp_vals, noisy_exp_vals

In [5]:
# train_circuits, train_ideal_exp_vals, train_noisy_exp_vals = load_circuits('./data/haoran_mbd/random_circuits/train/', '.pk')
# print(len(train_circuits))

test_circuits, test_ideal_exp_vals, test_noisy_exp_vals = load_circuits('./data/ising_init_from_qasm/val/', '.pk')
print(len(test_circuits))
test_noisy_exp_vals = [x[0] for x in test_noisy_exp_vals]

  0%|          | 0/15 [00:00<?, ?it/s]

1500


In [15]:
def get_zne_expval(circ,
                   obs: str,
                   extrapolator,
                   backend,
                   noise_factors = [1, 3, 5, 7],
                   amplifier=LocalFoldingAmplifier(gates_to_fold=2),
                   shots:int=10000,
                   ) -> float:
    observable = SparsePauliOp(obs)

    ZNEEstimator = zne(BackendEstimator)
    estimator = ZNEEstimator(backend=backend)

    zne_strategy = ZNEStrategy(
        noise_factors=noise_factors,
        noise_amplifier=amplifier,
        extrapolator=extrapolator
    )

    job = estimator.run(circ, observable, shots=shots, zne_strategy=zne_strategy)
    values = job.result().values
    # we use a different convention where +1 is the eigenvalue for |1> state, because cal_z_exp assumes 0 is -Z and 1 is +Z
    values *= -1

    return values.tolist()

def get_measurement_qubits(qc, num_qubit):
        measurement_qubits = []
        for measurement in range(num_qubit-1, -1, -1):
            measurement_qubits.append(qc.data[-1-measurement][1][0].index)
        return measurement_qubits

def zne_mitigating_single_z(circ, backend, extrapolator):
    mapping = get_measurement_qubits(circ, 4)
    mapping = mapping[::-1]
    for i in range(5):
        if i not in mapping:
            mapping = mapping + [i]

    mitigated = []
    # ob_list has single Z measurement on the first qubit in the last position, i.e., 'ZIIII' is measuring the top qubit (Non-endian notation)
    ob_list = np.array(['ZIIII', 'IZIII', 'IIZII', 'IIIZI', 'IIIIZ'][::-1])
    ob_list = ob_list[np.array(mapping)].tolist()   # ancilla always mapped to the last position of the ob_list
    for obs in ob_list[:-1]:
        mitigated += get_zne_expval(circ, obs, extrapolator, backend)
    return mitigated

zne_mitigating_single_z(test_circuits[1], backend_noisy, PolynomialExtrapolator(degree=2))

/var/folders/kj/mfqyg_tj77nf4gt67lvscdkw0000gn/T/ipykernel_92023/3930351170.py:30: DeprecationWarning: Back-references to from Bit instances to their containing Registers have been deprecated. Instead, inspect Registers to find their contained Bits.
  measurement_qubits.append(qc.data[-1-measurement][1][0].index)


[0.016817674684945362,
 0.016698986150298776,
 -0.0829414482139666,
 -0.11465554127456118]

In [26]:
import numpy as np
from concurrent.futures import ProcessPoolExecutor, as_completed
import multiprocessing
import dill
from multiprocessing import Pool

def get_measurement_qubits(qc, num_qubit):
        measurement_qubits = []
        for measurement in range(num_qubit-1, -1, -1):
            measurement_qubits.append(qc.data[-1-measurement][1][0].index)
        return measurement_qubits

def get_all_circuit_meas_mapping(circs):
    out = []
    for circ in circs:
        mapping = get_measurement_qubits(circ, 4)
        mapping = mapping[::-1]
        for i in range(5):
            if i not in mapping:
                mapping = mapping + [i]
        out.append(mapping)
    return out

OB_LIST = np.array(['ZIIII', 'IZIII', 'IIZII', 'IIIZI', 'IIIIZ'][::-1])
def get_all_circuit_ob_list(mappings):
    out = []
    for m in mappings:
        ob_list = OB_LIST[np.array(m)][:-1].tolist()
        out.append(ob_list)
    return out

def get_zne_expval_parallel_single_z(
        circs,
        extrapolator,
        backend,
        noise_factors = [1, 3, 5, 7],
        amplifier=LocalFoldingAmplifier(gates_to_fold=2),
        shots:int=10000,
) -> float:

    ZNEEstimator = zne(BackendEstimator)
    estimator = ZNEEstimator(backend=backend)

    zne_strategy = ZNEStrategy(
        noise_factors=noise_factors,
        noise_amplifier=amplifier,
        extrapolator=extrapolator
    )

    mappings = get_all_circuit_meas_mapping(circs)
    ob_list_all = get_all_circuit_ob_list(mappings)

    # mitigated = np.zeros((len(circs), 4))
    # for i, (circ, ob_list) in enumerate(zip(circs, ob_list_all)):
    #     ob_list = list(map(SparsePauliOp, ob_list))
    #     job = estimator.run([circ]*4, ob_list, shots=shots, zne_strategy=zne_strategy)
    #     values = job.result().values
    #     mitigated[i] = values

    return zne_strategy, estimator, ob_list_all, shots

extrapolator = PolynomialExtrapolator(degree=2)
zne_strategy, estimator, ob_list_all, shots = get_zne_expval_parallel_single_z(test_circuits[:100], extrapolator, backend_noisy)

##########################################################################################
def process_circ_ob_list(args):
    i, circ, ob_list = args
    ob_list = list(map(SparsePauliOp, ob_list))
    job = estimator.run([circ]*4, ob_list, shots=shots, zne_strategy=zne_strategy)
    values = job.result().values
    return i, values

def parallel_process(circs, ob_list_all):
    mitigated = np.zeros((len(circs), 4))
    iterable = [(i, circ, ob_list) for i, (circ, ob_list) in enumerate(zip(circs, ob_list_all))]
    if __name__ == '__main__':
        with Pool() as pool:
            results = pool.map(process_circ_ob_list, iterable)
        for i, values in results:
            mitigated[i] = values
    return mitigated

#####################################################
mitigated = -1 * parallel_process(test_circuits[:100], ob_list_all)

/var/folders/kj/mfqyg_tj77nf4gt67lvscdkw0000gn/T/ipykernel_92023/2446081229.py:10: DeprecationWarning: Back-references to from Bit instances to their containing Registers have been deprecated. Instead, inspect Registers to find their contained Bits.
  measurement_qubits.append(qc.data[-1-measurement][1][0].index)
Process SpawnPoolWorker-130:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/ngem/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/anaconda3/envs/ngem/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/envs/ngem/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/opt/anaconda3/envs/ngem/lib/python3.10/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'process_circ_ob_list' on <module '__main__' (built-in)>
Process SpawnPoolWorker-131:
T

KeyboardInterrupt: 

In [110]:
def zne_mitigating_single_z(circ, backend, extrapolator):


    mitigated = []
    # ob_list has single Z measurement on the first qubit in the last position, i.e., 'ZIIII' is measuring the top qubit (Non-endian notation)
    ob_list = np.array(['ZIIII', 'IZIII', 'IIZII', 'IIIZI', 'IIIIZ'][::-1])
    ob_list = ob_list[np.array(mapping)].tolist()   # ancilla always mapped to the last position of the ob_list
    for obs in ob_list[:-1]:
        mitigated += get_zne_expval(circ, obs, extrapolator, backend)
    return mitigated

zne_mitigating_single_z(test_circuits[k], backend_noisy, PolynomialExtrapolator(degree=2))

/var/folders/kj/mfqyg_tj77nf4gt67lvscdkw0000gn/T/ipykernel_89245/3097026354.py:36: DeprecationWarning: Back-references to from Bit instances to their containing Registers have been deprecated. Instead, inspect Registers to find their contained Bits.
  measurement_qubits.append(qc.data[-1-measurement][1][0].index)


[0.12761907610043433,
 -0.054354192330681374,
 0.12023924186408814,
 -0.05979693707610778]

In [ ]:
fix_random_seed(0)

distances = []

num_spins = 4
even_qubits = np.linspace(0, num_spins, int(num_spins/2), endpoint=False)
odd_qubits = np.linspace(1, num_spins+1, int(num_spins/2), endpoint=False)

degree = 2
extrapolator = PolynomialExtrapolator(degree=degree)

sl = slice(0, 100000)
for circ, ideal, noisy in tqdm(zip(test_circuits[sl], test_ideal_exp_vals[sl], test_noisy_exp_vals[sl]), total=len(test_circuits[sl])):
    ngm_mitigated = zne_mitigating_single_z(circ, backend_noisy, extrapolator)
    # to_print = np.zeros((3, 4))
    # to_print[0] = np.array(ideal)
    # to_print[1] = np.array(noisy)
    # to_print[2] = np.array(ngm_mitigated)
    # print(to_print)

    imbalance_ideal = calc_imbalance([ideal], even_qubits, odd_qubits)[0]
    imbalance_noisy = calc_imbalance([noisy], even_qubits, odd_qubits)[0]
    imbalance_mitigated = calc_imbalance([ngm_mitigated], even_qubits, odd_qubits)[0]
    for q in range(4):
        ideal_q = ideal[q]
        noisy_q = noisy[q]
        ngm_mitigated_q = ngm_mitigated[q]
        distances.append({
            f"ideal_{q}": ideal_q,
            f"noisy_{q}": noisy_q,
            f"ngm_mitigated_{q}": ngm_mitigated_q,
            f"dist_noisy_{q}": np.abs(ideal_q - noisy_q),
            f"dist_mitigated_{q}": np.abs(ideal_q - ngm_mitigated_q),
            f"dist_sq_noisy_{q}": np.square(ideal_q - noisy_q),
            f"dist_sq_mitigated_{q}": np.square(ideal_q - ngm_mitigated_q),
            "imb_ideal": imbalance_ideal,
            "imb_noisy": imbalance_noisy,
            "imb_ngm": imbalance_mitigated,
            "imb_diff": imbalance_ideal - imbalance_mitigated
        })

plt.style.use({'figure.facecolor':'white'})

df = pd.DataFrame(distances)

for q in range(4):
    print(f'RMSE_noisy_{q}:', np.sqrt(df[f"dist_sq_noisy_{q}"].mean()))
    print(f'RMSE_mitigated_{q}:', np.sqrt(df[f"dist_sq_mitigated_{q}"].mean()))

print(f'RMSE_noisy:', np.sqrt(np.mean([df[f"dist_sq_noisy_{q}"].mean() for q in range(4)])))
print(f'RMSE_mitigated:', np.sqrt(np.mean([df[f"dist_sq_mitigated_{q}"].mean() for q in range(4)])))

sns.boxplot(data=df[["dist_noisy_0", "dist_mitigated_0", "dist_noisy_1", "dist_mitigated_1", "dist_noisy_2", "dist_mitigated_2", "dist_noisy_3", "dist_mitigated_3"]], orient="h", showfliers = False)
plt.title("Dist to ideal exp value")
plt.show()

sns.histplot([df['ideal_0'], df['noisy_0'], df["ngm_mitigated_0"]], kde=True, bins=40)
plt.title("Exp values distribution")
plt.show()

random seed fixed to 0


  0%|          | 0/1500 [00:00<?, ?it/s]

/var/folders/kj/mfqyg_tj77nf4gt67lvscdkw0000gn/T/ipykernel_89245/3097026354.py:36: DeprecationWarning: Back-references to from Bit instances to their containing Registers have been deprecated. Instead, inspect Registers to find their contained Bits.
  measurement_qubits.append(qc.data[-1-measurement][1][0].index)
